In [5]:
import torch
from torch.autograd import Variable
import numpy as np
# rnn = torch.load('gender-rnn-classification.pt')

from data_util import *

In [2]:
import torch.nn as nn
from torch.autograd import Variable

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax()
    
    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def init_hidden(self):
        return Variable(torch.zeros(1, self.hidden_size,batch_size))

In [3]:
def evaluate(name_tensor):
    hidden = rnn.init_hidden()
    
    for i in range(name_tensor.size()[0]):
        output, hidden = rnn(name_tensor[i], hidden)
    
    return output

def predict(name, n_predictions=2):
    output = evaluate(Variable(name_to_tensor(name)))

    # Get top N categories
    topv, topi = output.data.topk(n_predictions, 1, True)
    predictions = []

    for i in range(n_predictions):
        value = topv[0][i]
        gender_index = topi[0][i]
        print('(%.2f) %s' % (value, all_genders[gender_index]))
        predictions.append([value, all_genders[gender_index]])

    return predictions

In [23]:
n_hidden = 128
n_epochs = 100000
print_every = 5000
plot_every = 1000
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn
batch_size = 16
visdom = False
num_workers = 2
start_iter = 0 # Begin counting iterations starting from this value (should be used with resume)
max_iter = 120000
stepvalues = (80000, 100000, 120000)

In [5]:
rnn = RNN(n_letters, n_hidden, n_genders)
optimizer = torch.optim.SGD(rnn.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()

In [6]:
dataset = NameGenderDataset(testset)
data_loader = data.DataLoader(dataset, 30, num_workers=2, shuffle=True, collate_fn=name_gender_collate)
batch_iterator = iter(data_loader)

In [7]:
names, genders = next(batch_iterator)

In [10]:
repr(len(valset))

'6782'

In [28]:
for name, gender in zip(names,genders):
    print(tensor_to_name(name), all_genders[gender.data[0]])

vittoriano male
eitelbert male
sigtraud female
semarias male
richelle female
mengquan male
hennadiy male
andriena female
volkart male
reyanna female
renbiao male
kenyaum male
sanura female
miylah female
masino male
jodice male
houwei male
gavina female
allart male
aimery male
uniya female
tyski male
sisko female
norio male
niesa female
lolan male
jacks male
frida female
dupre male
duice female


In [12]:
import string
ret = ""
for letter_tensor in names[19].split(1):
    print(letter_tensor)
    nz = letter_tensor.data.nonzero()
    if torch.numel(nz) != 0:
        ret += (string.ascii_lowercase[nz[0,1]])
print(ret)

Variable containing:

Columns 0 to 12 
    1     0     0     0     0     0     0     0     0     0     0     0     0

Columns 13 to 25 
    0     0     0     0     0     0     0     0     0     0     0     0     0
[torch.LongTensor of size 1x26]

Variable containing:

Columns 0 to 12 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 13 to 25 
    0     0     1     0     0     0     0     0     0     0     0     0     0
[torch.LongTensor of size 1x26]

Variable containing:

Columns 0 to 12 
    1     0     0     0     0     0     0     0     0     0     0     0     0

Columns 13 to 25 
    0     0     0     0     0     0     0     0     0     0     0     0     0
[torch.LongTensor of size 1x26]

Variable containing:

Columns 0 to 12 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 13 to 25 
    0     0     0     0     1     0     0     0     0     0     0     0     0
[torch.LongTensor of size 1x26]

Variable con

In [54]:
# torch.zeros(1,128)
# names[19].split(1)[0]
print('.' , end="")
print('.' , end="")
print('.' , end="")
print()
print("hello")

...
hello


In [34]:
torch.cat((names[19].split(1)[0], torch.zeros(1,128)), 0)

TypeError: cat received an invalid combination of arguments - got (tuple, int), but expected one of:
 * (sequence[torch.LongTensor] seq)
 * (sequence[torch.LongTensor] seq, int dim)
      didn't match because some of the arguments have invalid types: (!tuple!, int)


In [13]:
batch_size = 16
num_workers = 2
start_ep = 1 # Begin counting iterations starting from this value (should be used with resume)
end_ep = 100

In [59]:
num_ep = end_ep - start_ep + 1
len(dataset) % batch_size

8

In [105]:
mini = NameGenderDataset(split_dataset(0.9,0.0005)[1])
miniloader = data.DataLoader(mini, batch_size=4, num_workers=2,
                                  shuffle=True, collate_fn=name_gender_collate)

In [106]:
len(mini)

67

In [99]:
import progressbar
import logging
import time

In [96]:
bar = progressbar.ProgressBar()
for i in bar(range(100)):
    time.sleep(0.02)

In [100]:
progressbar.streams.wrap_stderr()
logging.basicConfig()

bar = progressbar.ProgressBar()
for i in bar(range(10)):
    logging.error('Got %d', i)
    time.sleep(0.2)

In [80]:
def run_mini():
    it = iter(miniloader)
    batch = 0
    while(True):
        try:
            batch += 1
            nm,gn = next(it)
            print("Batch #", batch, len(nm))
            for name in nm:
                print(tensor_to_name(name))
        except StopIteration:
            break

In [81]:
run_mini()

Batch # 1 4
charlottle
velizara
lawahiz
gilmore
Batch # 2 4
orbelin
katrina
brigitt
frimet
Batch # 3 4
ekkehardt
schuylar
jingtian
mayana
Batch # 4 1
janikqua


In [78]:
run_mini()

Batch # 1
ekkehardt
orbelin
lawahiz
mayana
Batch # 2
velizara
schuylar
janikqua
gilmore
Batch # 3
charlottle
jingtian
katrina
brigitt
Batch # 4
frimet


In [79]:
run_mini()

Batch # 1
charlottle
ekkehardt
janikqua
orbelin
Batch # 2
schuylar
gilmore
brigitt
mayana
Batch # 3
velizara
lawahiz
katrina
frimet
Batch # 4
jingtian


In [49]:
v = Variable(torch.rand(7,1,26))
v.long()

Variable containing:
(0 ,.,.) = 

Columns 0 to 8 
   0.7304  0.7241  0.7746  0.0598  0.3045  0.9179  0.9519  0.5221  0.9286

Columns 9 to 17 
   0.9225  0.5350  0.8277  0.3725  0.5689  0.7781  0.5558  0.1031  0.3205

Columns 18 to 25 
   0.8895  0.6413  0.3493  0.8204  0.4681  0.7177  0.5352  0.5811

(1 ,.,.) = 

Columns 0 to 8 
   0.3848  0.5293  0.2402  0.5740  0.7639  0.8949  0.5400  0.3663  0.6535

Columns 9 to 17 
   0.7766  0.8432  0.1905  0.7575  0.9952  0.1339  0.7068  0.1554  0.3810

Columns 18 to 25 
   0.7105  0.6054  0.9141  0.6224  0.1957  0.8083  0.9247  0.8236

(2 ,.,.) = 

Columns 0 to 8 
   0.9449  0.5020  0.1777  0.0459  0.5663  0.4822  0.5499  0.3416  0.9190

Columns 9 to 17 
   0.1089  0.7138  0.6793  0.7103  0.1771  0.2281  0.5325  0.2385  0.4749

Columns 18 to 25 
   0.4839  0.8185  0.5266  0.1435  0.4592  0.5884  0.2337  0.9653

(3 ,.,.) = 

Columns 0 to 8 
   0.0738  0.7432  0.7410  0.7047  0.8022  0.5676  0.2255  0.8328  0.0992

Columns 9 to 17 
   0.5631  0.59

In [51]:
torch.cat((v[0].float(),torch.zeros(1,128)),1)

Variable containing:

Columns 0 to 9 
 0.7304  0.7241  0.7746  0.0598  0.3045  0.9179  0.9519  0.5221  0.9286  0.9225

Columns 10 to 19 
 0.5350  0.8277  0.3725  0.5689  0.7781  0.5558  0.1031  0.3205  0.8895  0.6413

Columns 20 to 29 
 0.3493  0.8204  0.4681  0.7177  0.5352  0.5811  0.0000  0.0000  0.0000  0.0000

Columns 30 to 39 
 0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000

Columns 40 to 49 
 0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000

Columns 50 to 59 
 0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000

Columns 60 to 69 
 0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000

Columns 70 to 79 
 0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000

Columns 80 to 89 
 0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000

Columns 90 to 99 
 0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000

In [33]:
hidden = rnn.init_hidden()
optimizer.zero_grad()

for i in range(names.size(1)):
    output, hidden = rnn(names[:,i], hidden)

loss = criterion(output, genders)
loss.backward()

optimizer.step()

return output, loss.data[0]

TypeError: cat received an invalid combination of arguments - got (tuple, int), but expected one of:
 * (sequence[torch.LongTensor] seq)
 * (sequence[torch.LongTensor] seq, int dim)
      didn't match because some of the arguments have invalid types: (!tuple!, int)


In [25]:
names[:,1]

Variable containing:

Columns 0 to 12 
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     1     0     0     0     0     0
    0     0     0     0     0     0     0     0     1     0     0     0     0
    0     0     0     0     1     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 13 to 25 
    0     1     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     1     0     0     0     0     0
[torch.LongTensor of size 5x26]

In [34]:
hidden

Variable containing:
( 0 ,.,.) = 
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
     ...       ⋱       ...    
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
[torch.FloatTensor of size 1x128x16]

In [3]:
import test

usage: __main__.py [-h] [--batch_size BATCH_SIZE] [--num_workers NUM_WORKERS]
                   [--log_iters LOG_ITERS] [--weights WEIGHTS]
__main__.py: error: unrecognized arguments: -f /jupyter-runtime/kernel-c116fc5b-96ed-4782-ae2e-d6ec5b5f4748.json


SystemExit: 2

/usr/local/Anaconda3-4.4.0-Linux-x86_64/envs/jupyter-atp/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
test()

In [116]:
def time_since(since):
    now = time.time()
    s = now - since
    hours, rem = divmod(now-since, 3600)
    minutes, seconds = divmod(rem, 60)
    return "{:0>2}h {:0>2}m {:0>2}s".format(int(hours),int(minutes),int(seconds))

In [117]:
t = time.time()
time.sleep(1.55)
time_since(t)

'00h 00m 01s'

In [7]:
val = split_dataset(0.75,0.01)[1]

In [9]:
len(val)

1356

In [20]:
from model import RNN

In [24]:
print('Loading weights...')
rnn = RNN(n_letters, n_hidden, n_genders)
rnn.load_state_dict(torch.load('weights/gender_rnn_epoch21000.pth'))
rnn.eval()

Loading weights...


RNN (
  (i2h): Linear (154 -> 128)
  (i2o): Linear (154 -> 2)
  (softmax): LogSoftmax ()
)

In [25]:
def _evaluate(name_tensor):
    hidden = rnn.init_hidden()
    
    for letter_tensor in name_tensor:
        letter_tensor.data.unsqueeze_(0)
        output, hidden = rnn(letter_tensor, hidden)
    
    return output

In [30]:
def validate(dataset=valset):
    dataset = NameGenderDataset(dataset)
    data_loader = data.DataLoader(dataset, batch_size=batch_size, num_workers=num_workers,
                                  shuffle=True, collate_fn=name_gender_collate)
    
    # iterate over all minibatches
    start = time.time()
    batch_iterator = iter(data_loader)
    cum = 0
    while(True):
        try:
            print(time_since(start))
            names_tensor, genders_tensor = next(batch_iterator)
            for name_tensor, gender_tensor in zip(names_tensor,genders_tensor):
                gt = all_genders[gender_tensor.data[0]]
                name = tensor_to_name(name_tensor)
                output = _evaluate(name_tensor)
                topv, topi = output.data.topk(k=1, dim=1, largest=True)
                guess = all_genders[topi[0][0]]
                cum += 1 if guess == gt else 0
        except StopIteration:
            break
    acc = cum / len(dataset)
    return acc

In [31]:
validate(val)

00h 00m 00s
00h 00m 34s
00h 00m 50s
00h 01m 19s
00h 01m 52s
00h 02m 29s
00h 02m 59s
00h 03m 29s
00h 04m 04s
00h 04m 42s
00h 05m 11s
00h 05m 50s
00h 06m 20s
00h 06m 57s
00h 07m 31s
00h 08m 04s
00h 08m 37s
00h 09m 07s
00h 09m 46s
00h 10m 20s
00h 10m 49s
00h 11m 22s
00h 11m 41s
00h 12m 16s
00h 12m 49s
00h 13m 21s
00h 14m 09s
00h 14m 33s
00h 14m 59s
00h 15m 42s
00h 16m 20s
00h 16m 43s
00h 17m 11s
00h 17m 37s
00h 18m 12s
00h 18m 41s
00h 19m 15s
00h 19m 49s
00h 20m 19s
00h 20m 48s
00h 21m 15s
00h 21m 41s
00h 22m 10s
00h 22m 23s
00h 22m 53s
00h 23m 23s
00h 23m 44s
00h 24m 10s
00h 24m 51s
00h 25m 16s
00h 25m 51s
00h 26m 26s
00h 27m 02s
00h 27m 30s
00h 28m 24s
00h 28m 52s
00h 29m 16s
00h 29m 46s
00h 30m 17s
00h 30m 51s
00h 31m 14s
00h 31m 50s
00h 32m 29s
00h 32m 50s
00h 33m 14s
00h 33m 37s
00h 34m 05s
00h 34m 45s
00h 35m 18s
00h 35m 57s
00h 36m 33s
00h 37m 08s
00h 37m 40s
00h 38m 03s
00h 38m 34s
00h 39m 04s
00h 39m 39s
00h 40m 15s
00h 41m 05s
00h 41m 58s
00h 42m 27s
00h 43m 03s
00h 43m 31s
00h 

0.7315634218289085

In [32]:
validate(val[:16])

00h 00m 00s
00h 00m 07s


0.8125

In [34]:
len(trainset)

101730

In [35]:
len(testset)

33910